In [ ]:
'''
Issues:
    1. The output video as mirror images present 
    2. Accumulate with logAbs isn't working well 
    3.

To-do's:
    1. Implement windowing in both range and doppler processings 
    2. Folder selection
'''

In [28]:
# install required libraries
# for mmwave - pip install openradar

import numpy as np 
import time 
from mmwave.dsp.range_processing import *   # pip install openradar
from processing_chain import * 
import skvideo.io # pip install scikit-video
import matplotlib.pyplot as plt
import cv2
from config_radar import custom_config
import os
import glob
import tkinter as tk
from tkinter import filedialog


In [24]:
# radar specifications 
num_samples = 122
num_chirps = 128
num_tx = 1
num_rx = 4
frames_per_second = 20

header_size = 32

experiment_type = '_walk_parade'
time_now = time.ctime().lower()
str_time = time_now[4:7] + time_now[8:10] + '_' + time_now[11:13] + time_now[14:16] + '_'

In [25]:
# read all captured data and save into custom named directory
src_dir = '/mnt/c/work/rcube_extract/dca_capture/captured_data/'
captured_files = glob.glob(src_dir + '/*.*')

subdir_name = 'dca_' + str_time + 'trx_' + str(num_tx) + str(num_rx) + '_np_' + str(num_samples) + 'x' + str(num_chirps) 
subdir_name = subdir_name + '_fps' + str(frames_per_second) + experiment_type 

dst_dir = src_dir + subdir_name + '/'
os.mkdir(dst_dir)

for i in captured_files:
    dst_file = dst_dir + i.split("/")[-1]
    shutil.move(i, dst_file)


In [29]:
# open directory 
root = tk.Tk()
root.withdraw()

file_path = filedialog.askdirectory(initialdir='/mnt/c/work/rcube_extract/dca_capture/captured_data')
print(file_path)

# verify and update data parameters here 
num_samples = 122
num_chirps = 128
num_tx = 1
num_rx = 4
frames_per_second = 20

header_size = 32

TclError: no display name and no $DISPLAY environment variable

In [ ]:
# define video writing object 

frame_width = num_samples # //2 # To remove the part correpsonding to negative frequencies 
frame_height = num_chirps

save_path = '/mnt/c/work/rcube_extract/dca_capture/video_write/'
video_file = save_path + subdir_name + '.avi'

out = cv2.VideoWriter(video_file,cv2.VideoWriter_fourcc('M','J','P','G'), frames_per_second, (frame_width,frame_height), isColor=False)

In [ ]:
# read all the data files present in the folder
# Note: radarcube axes [fast time, slow time, channels, frames (time progression)]
file_index = 0 
prev_file_data = np.array([])

hsi_header = [2780, 3290, 2780, 3290,   48,    1,    0,    0, 1731,   32,  515,  512,  271,    1,  512,    0]
packet_len = num_samples *  num_rx * 2 + 32 

while True:
    try:
        raw_data = np.fromfile(fullfile.format(file_index), dtype=np.int16)
        raw_appended = np.insert(raw_data, 0, prev_file_data) #insert in the beginning

        # sanity check
        error1 = np.array_equal(raw_appended[0 : np.size(hsi_header)], hsi_header) == 0
        error2 = np.array_equal(raw_appended[packet_len:packet_len+np.size(hsi_header)], hsi_header) == 0
        if error1 or error2:
             print('Something wrong with data unwrapping!')

        rcube, next_frame_data = raw_radarcube(raw_appended, num_samples, num_chirps, num_rx, header_size)

        
        # range-doppler proessing of the raw data
        axis_range = 0
        axis_doppler = 1

        rcube_fft1 = range_processing(rcube, window_type_1d=None, axis=axis_range)
        rcube_fft2 = doppler_processing_custom(rcube_fft1, num_tx_antennas=1, clutter_removal_enabled=True, 
        interleaved=False, window_type_2d=None, accumulate=False, axis=axis_doppler)
        print('rcube dimensions file {0} : '.format(file_index) + str(rcube_fft2.shape))


        # write range-doppler output as video 
        
        # convert to proper format 
        rcube_fft2_tr = np.transpose(rcube_fft2, (1,0,2))
        # video_in = rcube_fft2_tr[:, :num_samples//2, :]   # to remove the part correpsonding to negative frequencies 
        video_in = rcube_fft2_tr
        # print(video_in.shape)

        # plot sample images 
        # plt.imshow(video_in[:,:,1])
        # plt.show()

        # normalize and write the frame to video object
        for frame in range(video_in.shape[2]):
            # img_normalize = cv2.normalize(src=video_in[:, :, frame], dst=None, alpha=0, beta=255, norm_type=cv2.NORM_L1, dtype=cv2.CV_8U)
            min = np.amin(video_in[:, :, frame], (0,1))
            max = np.amax(video_in[:, :, frame], (0,1))
            img_normalize = (video_in[:, :, frame] - min) / (max - min)
            img_normalize = np.round(img_normalize * 255).astype(np.uint8)

            out.write(img_normalize)
            if frame == 0 and file_index == 1:
                print(img_normalize)


    except IOError:
        print('file ' + filename.format(file_index) + ' does not exist!\n' + 'exiting file read ...\n')
        break
    finally:
        pass
        
    prev_file_data = next_frame_data     
    file_index += 1

# release the video object    
out.release()


In [ ]:
# read all the data files present in the folder
# Note: radarcube axes [fast time, slow time, channels, frames (time progression)]
file_index = 0 
prev_file_data = np.array([])

hsi_header = [2780, 3290, 2780, 3290,   48,    1,    0,    0, 1731,   32,  515,  512,  271,    1,  512,    0]
packet_len = num_samples *  num_rx * 2 + 32 

while True:
    try:
        raw_data = np.fromfile(fullfile.format(file_index), dtype=np.int16)
        raw_appended = np.insert(raw_data, 0, prev_file_data) #insert in the beginning

        # sanity check
        error1 = np.array_equal(raw_appended[0 : np.size(hsi_header)], hsi_header) == 0
        error2 = np.array_equal(raw_appended[packet_len:packet_len+np.size(hsi_header)], hsi_header) == 0
        if error1 or error2:
             print('Something wrong with data unwrapping!')

        rcube, next_frame_data = raw_radarcube(raw_appended, num_samples, num_chirps, num_rx, header_size)

        
        # range-doppler proessing of the raw data
        axis_range = 0
        axis_doppler = 1

        rcube_fft1 = range_processing(rcube, window_type_1d=None, axis=axis_range)
        rcube_fft2 = doppler_processing_custom(rcube_fft1, num_tx_antennas=1, clutter_removal_enabled=True, 
        interleaved=False, window_type_2d=None, accumulate=False, axis=axis_doppler)
        print('rcube dimensions file {0} : '.format(file_index) + str(rcube_fft2.shape))


        # write range-doppler output as video 
        
        # convert to proper format 
        rcube_fft2_tr = np.transpose(rcube_fft2, (1,0,2))
        # video_in = rcube_fft2_tr[:, :num_samples//2, :]   # to remove the part correpsonding to negative frequencies 
        video_in = rcube_fft2_tr
        # print(video_in.shape)

        # plot sample images 
        # plt.imshow(video_in[:,:,1])
        # plt.show()

        # normalize and write the frame to video object
        for frame in range(video_in.shape[2]):
            # img_normalize = cv2.normalize(src=video_in[:, :, frame], dst=None, alpha=0, beta=255, norm_type=cv2.NORM_L1, dtype=cv2.CV_8U)
            min = np.amin(video_in[:, :, frame], (0,1))
            max = np.amax(video_in[:, :, frame], (0,1))
            img_normalize = (video_in[:, :, frame] - min) / (max - min)
            img_normalize = np.round(img_normalize * 255).astype(np.uint8)

            out.write(img_normalize)
            if frame == 0 and file_index == 1:
                print(img_normalize)


    except IOError:
        print('file ' + filename.format(file_index) + ' does not exist!\n' + 'exiting file read ...\n')
        break
    finally:
        pass
        
    prev_file_data = next_frame_data     
    file_index += 1

# release the video object    
out.release()
